In [2]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" :"18AV1meKNbDmaIRYiqL2iG8XPLPovn8nCk5exIVOm5G5"
	}


In [3]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 4096,
    "repetition_penalty": 1
}

In [4]:
model_id = "sdaia/allam-1-13b-instruct"


In [5]:
project_id = "7c5c2625-4a94-4761-8385-ada9c69cf56d"
#space_id = os.getenv("SPACE_ID")


In [6]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	)


c:\Users\Roro1\anaconda3\Lib\site-packages\ibm_watsonx_ai\foundation_models\model.py:101: DeprecationWarning: The `Model` class is deprecated and will be removed in a future release. Please use the `ModelInference` class instead. To update your imports, use: `from ibm_watsonx_ai.foundation_models import ModelInference`.
  warn(


In [9]:
prompt_input = """انت مساعد شخصي للاعراب فقط اي سوال يجب ان يرد عليه بشرح للاعراب للاطفال"""


In [10]:
question = input("Question: ")
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")


AI:  كمساعد ذكاء صناعي، يمكنني مساعدتك في فهم قواعد النحو والإعراب بطريقة مبسطة وسهلة الفهم. إذا كان لديك سؤال حول إعراب جملة معينة أو كلمة، يرجى طرحها وسأقوم بشرح الإعراب بطريقة تناسب الأطفال.

على سبيل المثال، إذا كنت تريد معرفة إعراب كلمة "الكتاب" في الجملة "قرأت الكتاب"، سأشرح لك أن "الكتاب" هو اسم مجرور وعلامة جره الكسرة الظاهرة على آخره.

إذا كان لديك سؤال آخر أو تحتاج إلى مزيد من التوضيح، لا تتردد في طرحه وسأكون سعيداً لمساعدتك. 


In [11]:
from sentence_transformers import SentenceTransformer

model1=SentenceTransformer("intfloat/multilingual-e5-large")


In [ ]:
import os
import faiss
import numpy as np
import csv
from docx import Document

# Set the embedding dimension for the FAISS index
embedding_dim = 1024  # Replace with the correct embedding dimension of your model
index = faiss.IndexFlatL2(embedding_dim)  # Use L2 distance for similarity search
metadata = []

# Function to read the content of a DOCX file
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Function to split content into chunks with overlap
def split_into_chunks(text, chunk_size=512, overlap=50):
    tokens = text.split()  # Assuming words as tokens, modify if needed for other tokenization
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = " ".join(tokens[i:i + chunk_size])
        chunks.append(chunk)
        if i + chunk_size >= len(tokens):
            break
    return chunks

# Loop through all DOCX files in the current directory
for filename in os.listdir("."):
    if filename.endswith(".docx"):
        # Construct the file path
        file_path = os.path.join(".", filename)
        
        # Read the content of the DOCX file
        text = read_docx(file_path)

        # Split the document content into chunks
        chunks = split_into_chunks(text)

        # Process each chunk, generate embedding, and add it to FAISS index
        for chunk_num, chunk_text in enumerate(chunks):
            # Generate embeddings for the chunk's content
            embedding = model1.encode(chunk_text)

            # Reshape embedding to add it to the FAISS index
            embedding = np.array(embedding).reshape(1, -1)

            # Add the embedding to FAISS index
            index.add(embedding)

            # Store metadata for each chunk, including document name and chunk content
            metadata.append({
                "document_name": filename,
                "chunk_number": chunk_num + 1,  # Chunk numbering starts from 1
                "chunk_content": chunk_text,  # Store the actual text content of the chunk
                "embedding": embedding.tolist()
            })

# Save metadata to a CSV file
csv_file = "book_embedding.csv"
with open(csv_file, mode="w", newline='', encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["document_name", "chunk_number", "chunk_content", "embedding"])
    writer.writeheader()
    for entry in metadata:
        writer.writerow(entry)

# Save FAISS index to the project directory
faiss.write_index(index, "faiss_book_chunks.index")

print("Embeddings, FAISS index, and chunked content saved successfully.")


In [14]:
import faiss

index = faiss.read_index("faiss_book_chunks.index")


In [15]:
import pandas as pd

df = pd.read_csv('book_embedding.csv')

In [16]:
df.head()


,document_name,chunk_number,chunk_content,embedding
0,اختبار الوحدة الأولى الأرنب الذكي.docx,1,اختبار الوحدة الأولى الأرنب الذكي يُحكى أنّ أس...,"[[0.019123591482639313, -0.013785659335553646,..."
1,اختبار الوحدة الأولى الأرنب الذكي.docx,2,الَّذِي سَيَدُورُ بَيْنَ الأَرْنَبِ وَالحَيَوَ...,"[[0.0118407616391778, -0.0035381473135203123, ..."
2,الاختبار التشخيصي صندوق جدّتي.docx,1,الاختبار التشخيصي صندوق جدّتي في نهاية كلّ أسب...,"[[0.008984504267573357, -0.003333796514198184,..."
3,التواصل الكتابي بناء فقرتين موضوع .docx,1,التواصل الكتابي بناء فقرتين موضوع الحشرات منَ ...,"[[-0.011388594284653664, -0.007934107445180416..."
4,التواصل اللغوي - التواصل الشفهي.docx,1,التواصل اللغوي - التواصل الشفهي الدّرس الأوّل:...,"[[0.014624868519604206, -0.0006087979418225586..."


In [18]:
import gradio as gr
import numpy as np
from sklearn.preprocessing import normalize
def query_search_chapters(doc, query, k=1):
    # Filter DataFrame based on the specified document name
    filtered_df = df[df['document_name'] == doc]
    
    # Ensure that the FAISS index and filtered_df have the same indices
    indices_to_use = filtered_df.index.to_numpy()
    filtered_embeddings = np.array([index.reconstruct(int(idx)) for idx in indices_to_use])

    # Step 1: Generate the embedding for the query
    query_embedding = model1.encode([query])
    
    # Normalize the query embedding (FAISS uses cosine similarity which works with normalized vectors)
    query_embedding = normalize(query_embedding)

    # Step 2: Perform the search on the filtered embeddings (k is the number of nearest neighbors to retrieve)
    distances, indices = index.search(query_embedding, k)

    # Step 3: Retrieve metadata for the top k results from the filtered DataFrame
    results = []
    for i in range(min(k, len(indices[0]))):  # Ensure we don't exceed the index bounds
        idx = indices[0][i]
        
        # Check if the index is within bounds
        if idx >= len(indices_to_use):
            continue  # Skip if index is out of bounds
        
        # Map FAISS index to the original index in `df`
        original_idx = indices_to_use[idx]
        
        # Retrieve metadata for the corresponding chunk
        result = filtered_df.loc[original_idx]
        result['similarity'] = distances[0][i]  # Add the similarity score (distance)
        results.append(result)
    
    return results

# Define the function that Gradio will use, with the document name set by default
def generate_grammar_explanation(question):
    document_name = "مراجعة السابقة المكتسبات.docx"  # Set the default document name
    prompt_input = "انت مساعد شخصي فقط اي سوال يجب ان يرد عليه بشرح للاعراب للاطفال"
    
    # Perform the search using the query and document name
    top_k_results = query_search_chapters(document_name, question, k=2)
    
    # Concatenate the most relevant chunks into the prompt
    context = ""
    for result in top_k_results:
        context += f"النص: {result['chunk_content']}\n"
    
    # Prepare the full prompt with context and the question
    formattedQuestion = f"<s> [INST] {question} [/INST]"
    prompt = f"{prompt_input}\n{context}\n{formattedQuestion}"

    # Generate response using the model
    generated_response = model.generate_text(prompt=prompt, guardrails=False)
    return generated_response

# Set up Gradio interface
iface = gr.Interface(
    fn=generate_grammar_explanation,
    inputs=gr.Textbox(label="Question"),
    outputs=gr.Textbox(label="Answer"),
    title="Arabic Grammar Explanation for Children",
    description="Ask any Arabic grammar question, and get an easy-to-understand explanation tailored for kids."
)

# Launch the interface
iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://2a07ecd7676a81f0f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [23]:
import gradio as gr
import numpy as np
from sklearn.preprocessing import normalize
# Function to perform the search
def query_search(query, k=1):
    # Step 1: Generate the embedding for the query
    query_embedding = model1.encode([query])
    
    # Normalize the query embedding (FAISS uses cosine similarity which works with normalized vectors)
    query_embedding = normalize(query_embedding)
    
    # Step 2: Perform the search with FAISS (k is the number of nearest neighbors to retrieve)
    distances, indices = index.search(query_embedding, k)  # distances and indices of the closest neighbors
    
    # Step 3: Retrieve metadata for the top k results
    results = []
    for i in range(k):
        # Get the index and metadata from the CSV file using the index from FAISS
        idx = indices[0][i]
        result = df.iloc[idx]  # Access the metadata for the chunk
        result['similarity'] = distances[0][i]  # Add the similarity score (distance)
        results.append(result)
    
    return results

# Define the function that Gradio will use, with the document name set by default
def generate_grammar_explanation(question):
    prompt_input = "انت مساعد شخصي فقط اي سوال يجب ان يرد عليه بشرح للاعراب للاطفال"
    
    # Perform the search using the query and document name
    top_k_results = query_search(question, k=2)
    
    # Concatenate the most relevant chunks into the prompt
    context = ""
    for result in top_k_results:
        context += f"النص: {result['chunk_content']}\n"
    
    # Prepare the full prompt with context and the question
    formattedQuestion = f"<s> [INST] {question} [/INST]"
    prompt = f"{prompt_input}\n{context}\n{formattedQuestion}"

    # Generate response using the model
    generated_response = model.generate_text(prompt=prompt, guardrails=False)
    return generated_response

# Set up Gradio interface
iface = gr.Interface(
    fn=generate_grammar_explanation,
    inputs=gr.Textbox(label="Question"),
    outputs=gr.Textbox(label="Answer"),
    title="Arabic Grammar Explanation for Children",
    description="Ask any Arabic grammar question, and get an easy-to-understand explanation tailored for kids."
)

# Launch the interface
iface.launch(share=True)


c:\Users\Roro1\anaconda3\Lib\site-packages\pydantic\_internal\_generate_schema.py:330: RuntimeWarning: coroutine 'Server.serve' was never awaited
  def __init__(self, types_namespace: TypesNamespace):


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://0a1133f6c707500130.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


2oZTQ5C6H8YErAIomyO38C6HLM3_3m5wNjEU6ZtgnNz74iSJ3

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from flask_cors import CORS
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
from sklearn.preprocessing import normalize

# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes and origins

CORS(app, resources={r"/*": {"origins": "*", "methods": ["POST", "OPTIONS"]}})

# Set ngrok API key
ngrok.set_auth_token('')

run_with_ngrok(app)

# Function to perform query search using FAISS
def query_search(query, k=1):
    query_embedding = model1.encode([query])  # Generate embedding for the query
    query_embedding = normalize(query_embedding)  # Normalize embedding
    
    distances, indices = index.search(query_embedding, k)  # Search in the index
    results = []
    
    for i in range(k):
        idx = indices[0][i]  # Retrieve the index
        result = df.iloc[idx]  # Access corresponding chunk content
        result['similarity'] = distances[0][i]  # Add similarity score
        results.append(result)
    
    return results

# Define Flask route for generating grammar explanation
@app.route("/generate_grammar_explanation", methods=["POST"])
def generate_grammar_explanation():
    data = request.get_json()
    question = data.get("question")
    
    # Search for relevant content based on the question
    top_k_results = query_search(question, k=2)
    
    context = ""
    for result in top_k_results:
        context += f"النص: {result['chunk_content']}\n"
    
    formattedQuestion = f"<s> [INST] {question} [/INST]"
    prompt = f"انت مساعد شخصي فقط اي سوال يجب ان يرد عليه بشرح للاعراب للاطفال\n{context}\n{formattedQuestion}"
    
    # Generate response using the model
    generated_response = model.generate_text(prompt=prompt, guardrails=False)
    
    return jsonify({"answer": generated_response})

if __name__ == "__main__":
    app.run()


c:\Users\Roro1\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
